In [1]:
from os import stat
import matplotlib.pyplot as plt
import numpy as np
import json
import toml
import matplotlib.animation as anim
from IPython.display import HTML
plt.matplotlib.use("TkAgg")

def load_sim_states(fname):
    """Load states during simulation."""
    with open(fname, "r") as f:
        states = []
        for line in f:
            states.append(json.loads(line))
        return states
        
def load_toml(fname = "../zebra.toml"):
    """Load config toml."""
    with open(fname, "r") as f:
        parsed_toml = toml.loads(f.read())
    return parsed_toml

# Load zebra toml
zebra_toml = load_toml()

# Load states
states = load_sim_states("../sim_states.json")

# Testing
# state = states[100]
# ped_xs = [zebra_toml["zebra_crossings"][int(ped["location"])] for ped in state["pedestrians"]]
# veh_xs = [float(vehicle["position"]) for vehicle in state["vehicles"]]


In [2]:
# Function to create animation
def get_animation(states):
    fig = plt.figure(figsize=(10, 4))  
    ax1 = fig.add_subplot(1,1,1)
    GAP = 0.1
    WIDTH = 0.4
    LENGTH = 10
    N_STRIPES = 7
    MAX_LENGTH = zebra_toml["road_length"] + 1
    ped_scat, = ax1.plot([], [], color="dodgerblue", marker="o", markersize=9, ls="None")
    veh_scat, = ax1.plot([], [], marker=">", color="firebrick", ms=5, ls="None")
    ax1.set_xlim(0, MAX_LENGTH)
    min_y, max_y = -1, 1
    ax1.set_ylim(min_y, max_y)
    ax1.axis("off")
    ax1.vlines(np.arange(0, MAX_LENGTH, 100), min_y, max_y, ls=":", color="grey", lw=.5, zorder=-1)
    for crossing in zebra_toml["zebra_crossings"]:
        ax1.hlines(np.linspace(-WIDTH, WIDTH, N_STRIPES), crossing, crossing+LENGTH, color="k", lw=5, zorder=-4)
    
    def animate(i): 
        state = states[i]
        ped_xs = [zebra_toml["zebra_crossings"][int(ped["location"])]+LENGTH/2 for ped in state["pedestrians"]]
        ped_ys = []
        counts = {}
        for el in ped_xs:
            if el in counts:
                counts[el] += GAP
            else:
                counts[el] = GAP + WIDTH
            ped_ys.append(counts[el])

        veh_xs = [float(vehicle["position"]) for vehicle in state["vehicles"]]
        veh_ys = len(veh_xs)*[0]
        ped_scat.set_data(ped_xs, ped_ys)
        veh_scat.set_data(veh_xs, veh_ys)
        return (ped_scat, veh_scat)

    return anim.FuncAnimation(fig, animate, frames=np.arange(0, min(500, len(states))),
                              repeat=False, blit=True, interval=150)

In [3]:
a = get_animation(states)
HTML(a.to_html5_video())